# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 

#Abrindo navegador
navegador = webdriver.Chrome();
url = "https://google.com/"

#Passo 1: Pegar a cotação do dólar
#entrar no site
navegador.get(url)
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao dolar");
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER);
#pegar valor da cotação
cotacaodolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value');
display(cotacaodolar);

#Passo 2: Pegar a cotação do euro
navegador.get(url)
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao euro");
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER);
#pegar valor da cotação
cotacaoeuro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value');
display(cotacaoeuro);

#Passo 3: Pegar a cotação do ouro
navegador.get("https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiv8bWju8P8AhX-p5UCHWqxBJUQFnoECAkQAQ&url=https%3A%2F%2Fwww.melhorcambio.com%2Fouro-hoje&usg=AOvVaw1rJuZ4Mfu85WLH7Xeb8Tvf")
cotacaoouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacaoouro = cotacaoouro.replace(",",".")
display(cotacaoouro);

navegador.quit()


'5.0961'

'5.527484865'

'315.12'

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [6]:

#Passo 4:Importar a base de dados e Atualiza-la
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [7]:
#Passo 5: Recalcular os preços

#atualizar a cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacaodolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacaoeuro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacaoouro)

#recalcular os preços

#preço de compra = cotação * preço original
tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

#preço de venda = preço de compra * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.096100,5096.049039,1.40,7134.468655
1,Carro Renault,4500.00,Euro,5.527485,24873.681893,2.00,49747.363785
2,Notebook Dell,899.99,Dólar,5.096100,4586.439039,1.70,7796.946366
3,IPhone,799.00,Dólar,5.096100,4071.783900,1.70,6922.032630
4,Carro Fiat,3000.00,Euro,5.527485,16582.454595,1.90,31506.663730
5,Celular Xiaomi,480.48,Dólar,5.096100,2448.574128,2.00,4897.148256
6,Joia 20g,20.00,Ouro,315.120000,6302.400000,1.15,7247.760000


### Agora vamos exportar a nova base de preços atualizada

In [8]:
#Passo 6: Exportar a base de dados Atualizada

tabela.to_excel("Produtos Novos.xlsx")